## Text Summarization Using Langchain 

In [80]:
import os 
open_api_key="-C226Knva0CRxlCxdKbE2T3BlbkFJiEtS6mJC5v6n4erT1i7x"
os.environ["OPENAI_API_KEY"]=open_api_key

In [75]:
## Basic Prompt Summarization
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [76]:
speech="""
People across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you."""

print(len(speech))

439


In [77]:
chat_messages=[
    SystemMessage(content='You are an expert assistant with expertize in summarizing speeches'),
    HumanMessage(content=f'Please provide a short and concise summary of the following speech:\n TEXT: {speech}')
]

llm=ChatOpenAI(model_name='gpt-3.5-turbo')


In [78]:

llm.get_num_tokens(speech)

98

In [79]:
llm(chat_messages).content

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
##get_summary
content = llm(chat_messages).content
print(content)

In [ ]:
print(len(content))

## Prompt Templates Text Summarization

In [ ]:

from langchain.chains import LLMChain
from langchain import PromptTemplate

In [ ]:
generic_template='''
Write a summary of the following speech:
Speech : `{speech}`
Translate the precise summary to {language}.

'''
prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generic_template
)

In [ ]:
prompt.format(speech=speech,language='Hindi')

In [ ]:
complete_prompt=prompt.format(speech=speech,language='Urdu')

In [ ]:
llm.get_num_tokens(complete_prompt)

In [ ]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run({'speech':speech,'language':'urdu'})

In [ ]:
summary


In [ ]:
len(summary)

## StuffDocumentChain Text Summarization

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('apj.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [ ]:
text
pdf_list = []
pdf_list.append(text)
text

In [ ]:
from langchain.docstore.document import Document

In [ ]:
docs = [Document(page_content=text)]

In [ ]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [ ]:
template = '''Write a concise and short summary of the following speech.
Speech: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [ ]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

In [ ]:
output_summary

## Summarizing Large Documents Using Map Reduce

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('apjspeech.pdf')
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [ ]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [ ]:
llm.get_num_tokens(text)

In [ ]:
## Splittting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [ ]:
len(chunks)

In [ ]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
summary = chain.run(chunks)

In [ ]:
summary

## Map Reduce With Custom Prompts

In [ ]:
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [ ]:
final_combine_prompt='''
Provide a final summary of the entire speech with these important points.
Add a Generic Motivational Title,
Start the precise summary with an introduction and provide the
summary in number points for the speech.
Speech: `{text}`
'''
final_combine_prompt_template=PromptTemplate(input_variables=['text'],
                                             template=final_combine_prompt)

In [ ]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [ ]:
output

## RefineChain For Summarization

In [ ]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)

In [ ]:
output_summary